# Part 3 - LLM as Judge
- Use proprietary LLM to assess domain names created by fine tuned llm using OpenAI API
- This finishes off the pipeline and utlizes a rubric to assess the names (in addition to the BLEU/ROUGE metrics we used already)
- Note that we only consider valid domain names (which we pruned when fine tuning)

In [1]:
!pip install -q openai

In [2]:
import os
import openai
from openai import OpenAI
import json
import pandas as pd
from tqdm import tqdm

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
#openai.api_key = user_secrets.get_secret("openai_api_key")
api_key = user_secrets.get_secret("openai_api_key")

In [3]:
def format_prompt(description: str, domain_name: dict) -> str:
    return f"""
You are an expert branding consultant. Your task is to evaluate the quality of a proposed domain name for a business.

Business Description:
\"\"\"{description}\"\"\"

Proposed Domain Name:
\"\"\"{domain_name}\"\"\"

Evaluate the domain on the following criteria, providing scores from 0-5 for each criteria:
1. **Relevance**: Does the domain name relate well to the business?
2. **Creativity**: Is the name original and imaginative?
3. **Brandability**: Is it catchy, easy to remember, and easy to spell?
4. **Safety**: Is the name appropriate and free of offensive language?

Provide a JSON response like:
{{
  "relevance": 4,
  "creativity": 5,
  "brandability": 4,
  "safety": 5,
  "comments": "Creative and brandable. Slightly generic but safe and relevant."
}}

ONLY respond with the JSON and nothing else.
"""

In [4]:
# Initialize OpenAI client
client = OpenAI(api_key=api_key)

def call_judge(prompt):
    #prompt = format_prompt(description, domain_name)

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )

    content = response.choices[0].message.content.strip()
    try:
        parsed = json.loads(content)
    except json.JSONDecodeError:
        parsed = eval(content)  # fallback (not recommended)
        return None
    return parsed

In [5]:
def run_moderation_check(prompt):
    try:
        response = client.moderations.create(
            model="omni-moderation-latest",
            input=prompt
        )
        result = response.results[0]
        flagged = result.flagged
        categories = {k: v for k, v in result.categories.items() if v}
        return flagged, categories
    except Exception as e:
        print(f"Moderation check failed: {e}")
        return False, {}

In [6]:
def evaluate_domains(pred_csv_path, output_path):
    df = pd.read_csv(pred_csv_path)
    evaluations = []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        if not row['is_valid_domain']:
            continue
        prompt = format_prompt(row["description"], row["pred"])
        result = call_judge(prompt)

        eval_data = {**row}
        if result:
            eval_data.update(result)
        else:
            eval_data.update({
                "relevance": None,
                "creativity": None,
                "brandability": None,
                "safety": None,
                "comments": "Failed to evaluate"
            })

        # Run safety check using Moderation API
        flagged, categories = run_moderation_check(prompt)
        eval_data.update({
            "moderation_flagged": flagged,
            "moderation_categories": "; ".join(categories.keys()) if flagged else ""
        })

        evaluations.append(eval_data)

    
    pd.DataFrame(evaluations).to_csv(output_path, index=False)
    print(f"Saved evaluations to {output_path}")

In [7]:
# Run the evaluations 
version = 'v2'
flag = 'small'
input_file = f'/kaggle/input/domain-name-generator/data/predictions_eval-{version}-{flag}.csv'
output_file = f'judged_domains-{flag}-{version}.csv'
evaluate_domains(input_file, output_file)

  5%|▌         | 1/20 [00:05<01:39,  5.23s/it]

Moderation check failed: 'Categories' object has no attribute 'items'


 10%|█         | 2/20 [00:11<01:48,  6.05s/it]

Moderation check failed: 'Categories' object has no attribute 'items'


 15%|█▌        | 3/20 [00:15<01:27,  5.16s/it]

...

Moderation check failed: 'Categories' object has no attribute 'items'


 95%|█████████▌| 19/20 [01:05<00:03,  3.59s/it]

Moderation check failed: 'Categories' object has no attribute 'items'


100%|██████████| 20/20 [01:10<00:00,  3.55s/it]

Moderation check failed: 'Categories' object has no attribute 'items'
Saved evaluations to judged_domains-small-v2.csv
